In [8]:
import pathlib
import marko
import bs4
import re

from string import Template

NOTES_ROOT = 'notes'

def write_file(path, data, mode='w+'):
    path = pathlib.Path(path)
    try:
        with open(path.resolve().as_posix(), mode) as f:
            f.write(data)
    except (Exception,):
        print('Err.', 'write_file: try: with open()')
        return

    return path.as_posix()

def read_file(path: str | pathlib.Path):
    path = pathlib.Path(path)

    try:
        with open(path.resolve().as_posix()) as f:
            data = f.read()
    except (Exception,):
        print('Err - except Exception: try: with open(path.absolute().as_posix()) ')
        return

    return data



def oneNote(path):

    parts = {
        'title': '',
        'link': '',
        'description': '',
        'thumbnail': ''
    }
    
    data = read_file(path)
    markdown = marko.Markdown()
    html = markdown(data)
    
    soup = bs4.BeautifulSoup(html, 'html.parser')

    if titles := soup.find_all(re.compile("^h[1-6]$")):
        parts['title'] = titles[0].text  
    
    if youtubelink_s := soup.find_all('a', href=True):
        parts['url'] = youtubelink_s[0]['href'] 

    if youtubelink_s := soup.find_all('a', href=True):
        if len(youtubelink_s) > 1:
            parts['url_short'] = youtubelink_s[1]['href'] 
    
    if description_s := soup.find_all('code'):
        parts['description'] = description_s[0].text.strip()
    
    if thumbnail_s := soup.find_all('img', src=True):
        parts['thumbnail'] = thumbnail_s[0]['src']

    return parts
    



cardTemplate = Template('''
<div class="card m-1 mb-4 " >
    <img class="card-img-top lazy" src="$thumbnail" >
    <div class="card-body">
        <h5 class="card-title">$title</h5>
        
        <p class="card-text">
            $description
        </p>
        <p class="card-text">
            <a href="$url_short" class="stretched-link">$url_short</a>
        </p>
    </div>
</div>
''')

cardsTemplate = Template('''
<div class="card-columns">
$content
</div>
''')





def notesGrid():
    notes = pathlib.Path().absolute().parent.joinpath(NOTES_ROOT)

    files = list(notes.iterdir())
    mds = list(filter(lambda ff: ff.suffix == '.md', files))

    cards_html = ''
    
    for md in mds:
        print(md)
        parts = oneNote(md)

        mapping = {
            'title': parts['title'], 
            'url':  parts['url'],
            'url_short':  parts['url_short'],
            'description': parts['description'],
            'thumbnail': parts['thumbnail']
        }
        card_html = cardTemplate.substitute(**mapping).strip()
    
        cards_html += f'{card_html}\n'


    cardsHtml = cardsTemplate.substitute(**{'content': cards_html})

    htmlBase = Template(read_file('baseTemplate.html')).substitute(**{'content': cardsHtml})
    name = notes.joinpath('index.html')
    write_file(name, htmlBase)
    
notesGrid()
    

/Users/andrewlevin/Desktop/practice-github-cations/notes/1695259041_a-filonenko-kurs-pinokkio-vveden_ErNBeDutHR0.md
/Users/andrewlevin/Desktop/practice-github-cations/notes/1695259038_andrei-zubov-tainaia-vecheria_F8cu1MQfrq4.md
/Users/andrewlevin/Desktop/practice-github-cations/notes/1695259057_a-akhutin-kurs-evropeiskii-chelo_lZKWZ23vmSM.md
/Users/andrewlevin/Desktop/practice-github-cations/notes/1695259049_kak-zhit-svoei-zhizniu-aleksandr_g0dzarXiZoM.md
/Users/andrewlevin/Desktop/practice-github-cations/notes/1695259055_aleksandr-filonenko-kosmologiia-_T9R57NTJYKY.md
/Users/andrewlevin/Desktop/practice-github-cations/notes/1695259043_27-iz-33-iurii-lotman-iskusstvo-_KmxFLGRhg2c.md
/Users/andrewlevin/Desktop/practice-github-cations/notes/1695259053_otkrytaia-lektsiia-r-r-vakhitova_dDAFn-JIrOU.md
/Users/andrewlevin/Desktop/practice-github-cations/notes/1695259051_t-levina-oslepitelnyi-mrak-chern_e0ndxj-ARcg.md
/Users/andrewlevin/Desktop/practice-github-cations/notes/1695259047_semioti